In [ ]:
import ipywidgets as widgets
from IPython.display import display
import subprocess

In [ ]:
# Define available options for each key
os_versions = {'Ubuntu 22.04 Jammy': 'ubuntu/images/*ubuntu-jammy-22.04-amd64-server-*', 'Ubuntu 16.04 Xenial': 'ubuntu/images/*ubuntu-xenial-16.04-amd64-server-*'}
instance_types = {'Micro': 't2.micro', 'Small': 't2.small', 'Large': 't2.large'}
regions = {'US East': 'us-east-1', 'US West': 'us-west-1', 'Europe': 'eu-central-1'}

# Create checkbox widgets for each key
os_checkbox = widgets.RadioButtons(
    options=os_versions,
    description='OS version:'
)

instance_checkbox = widgets.RadioButtons(
    options=instance_types,
    description='Instance Type:'
)

region_checkbox = widgets.RadioButtons(
    options=regions,
    description='Region:'
)

# Display the checkboxes
display(os_checkbox)
display(instance_checkbox)
display(region_checkbox)


In [ ]:
# Collect the choices into a dictionary
def collect_choices():
    img_choices = {
        'OS version': os_checkbox.value,
        'Instance Type': instance_checkbox.value,
        'Region': region_checkbox.value
    }
    return img_choices

# Example usage:
image_choices = collect_choices()
print("User's Choices:", image_choices)

In [ ]:
# Create radio buttons for user choice
main_choice = widgets.RadioButtons(options=["Jupyter Notebook", "Run Code"], description="Choose:")


# Display radio buttons
display(main_choice)


In [ ]:
# Assigning dictionary values
if main_choice.value == "Jupyter Notebook":
    
    choices = {
        'Python Kernel': True,
        'R Kernel': False
    }
    python_checkbox = widgets.Checkbox(value=True, description="Python Kernel", disabled=True)
    r_checkbox = widgets.Checkbox(value=False, description="R Kernel")
else:
    choices = {
        'Python': False,
        'R': False
    }
    python_checkbox = widgets.Checkbox(value=False, description="Python")
    r_checkbox = widgets.Checkbox(value=False, description="R")

print(main_choice.value)
print(choices)
display(python_checkbox)
display(r_checkbox)

In [ ]:
# Change dict values based on user input
if main_choice.value == 'Jupyter Notebook':
    if python_checkbox.value:
        choices["Python Kernel"] = True
    else:
        choices["Python Kernel"] = False

    if r_checkbox.value:
        choices["R Kernel"] = True
    else:
        choices["R Kernel"] = False
else:
    if python_checkbox.value:
        choices["Python"] = True
    else:
        choices["Python"] = False

    if r_checkbox.value:
        choices["R"] = True
    else:
        choices["R"] = False

print(main_choice.value)
print(choices)

In [ ]:
# Create a dict for storing shell commands needed for the provisioner script
command_map = {
    'Python Kernel': '''\
sudo apt-get install -y python3 python3-pip
pip install jupyter

source ~/.profile

jupyter notebook --generate-config
echo "c.NotebookApp.ip = '0.0.0.0'" | tee -a /home/terraform/.jupyter/jupyter_notebook_config.py
echo "c.NotebookApp.open_browser = False" | tee -a /home/terraform/.jupyter/jupyter_notebook_config.py
    ''',
    'R Kernel': '''\
sudo apt -y install r-base
sudo R -e 'install.packages("IRkernel")'

R --no-save << EOF2
IRkernel::installspec()
q()
EOF2
    '''
}

In [ ]:
# Read code from provided path and store in command_map along with other commands
def read_code_from_file(file_path):
    with open(file_path, 'r') as file:
        code = file.read()
    return code

if main_choice.value == 'Run Code':
    if choices['Python']:
        file_path = input('Input the file path for your Python code: ')
        py_code = read_code_from_file(file_path)
        command_map['Python'] = f'''\
sudo apt-get install -y python3 python3-pip
python3 << EOF2 > output_py.txt
{py_code}
EOF2
'''
    if choices['R']:
        file_path = input('Input the file path for your R code: ')
        r_code = read_code_from_file(file_path)
        command_map['R'] = f'''\
sudo apt -y install r-base
R --no-save << EOF3 > output_r.txt
{r_code}
q()
EOF3
''' 

In [ ]:
commands = [command_map[choice] for choice, selected in choices.items() if selected]

# Open setup.sh in append mode and write the commands
with open('../scripts/setup.sh', 'a') as f:
    f.write('\n')  # Add a newline for clarity
    for command in commands:
        f.write(command + '\n')
    f.write('EOF\n')  # Add EOF marker after the last command

In [ ]:
# Create the packer config by passing choices too the script as arguments
subprocess.run(['../scripts/create_packer.sh', 'image.pkr.hcl', image_choices['Instance Type'], image_choices['Region'], image_choices['OS version']])

In [ ]:
# Initialize packer image
subprocess.run('packer init ../images/image.pkr.hcl', shell=True)

In [ ]:
# Build packer image
subprocess.run('packer build ../images/image.pkr.hcl', shell=True)

In [ ]:
# Parse the created manifest.json file for the AMI ID and pass it to a variable
result = subprocess.run('jq -r \'.builds[0].artifact_id|split(":")[1]\' ./manifest.json', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
ami_id = result.stdout.strip()
print(ami_id)

In [ ]:
# Create tfvars file which stores variables for the Terraform config
subprocess.run(['../scripts/create_tfvars.sh', image_choices['Region'], ami_id, image_choices['Instance Type']])

In [ ]:
# Deploy the image using Terraform
subprocess.run('terraform -chdir=../instances init && terraform -chdir=../instances apply -auto-approve', shell=True)